In [1]:
import pandas as pd

#### MAKE SCORE FILE

def make_score(df): 
    small = df[["Genotype", "REF", "ALT", "beta"]].copy()
    small_bad = small[small["beta"] < 0 ] 
    small_good = small[small["beta"] > 0 ] 
    small_bad["weight"] = small_bad["beta"] * -1 
    small_bad["A1"] = small_bad["REF"] 
    small_bad = small_bad[["Genotype", "A1", "weight"]].copy()
    small_good["weight"] = small_good["beta"]
    small_good["A1"] = small_good["ALT"]
    small_good = small_good[["Genotype", "A1", "weight"]].copy()
    SCORE = pd.concat([small_good, small_bad], 0 ) 
    SCORE["Genotype"] = SCORE["Genotype"].str.replace("-",":")
    return(SCORE)

###### BASE FUNCTION TO DO CLUMPING BASED ON VARIANT CLASS
import warnings
warnings.filterwarnings('ignore')


def normal_clump(df): 
    df = df.sort_values(["CHR","BP"])
    df = df.drop_duplicates(subset=['Genotype'], keep='first') ##### This is added as same protein/genotype across different array types
    print ("Number of variants before clump: " + str(df.shape[0]) ) 
    remove_list = []
    list1 = df.Genotype.to_list()
    list2 = list1[::-1]
    for i in list1: 
        for b in list2: 
            if i == b: 
                next
            if i != b: 
                i_chr =  df[df["Genotype"] == i ].CHR.values
                b_chr =  df[df["Genotype"] == b ].CHR.values
                i_bp =  df[df["Genotype"] == i ].BP.values
                b_bp =  df[df["Genotype"] == b ].BP.values
                i_group =  df[df["Genotype"] == i ]["p-value"].values
                b_group =  df[df["Genotype"] == b ]["p-value"].values

                if i_chr == b_chr : 
                    closeby = abs(int(b_bp) - int(i_bp))
                    if closeby < 10000000 : 
                        i2 = i.replace("-", ":")
                        b2 = b.replace("-", ":")
                        try: 
                            R = ld_matrix[(ld_matrix["SNP_A"] == i2 ) & (ld_matrix["SNP_B"] == b2 )].R2.values 
                        except: 
                            R = 0
                        if R > 0.05 : 
                            if i_group > b_group: 
                                remove_list.append(i) 
                            elif i_group < b_group:
                                remove_list.append(b) 
                            else: 
                                remove_list.append(i)
    temp_df = df[~df["Genotype"].isin(remove_list) ] 
    print ("Number of variants after clump: " + str(temp_df.shape[0]))
    return temp_df 

In [44]:
### add something
#raw = pd.read_csv("ewas_results_genomodel.txt", sep="\t")
raw = pd.read_excel("exome_gwas.xlsx" ) 
proteins_of_interest = pd.read_csv("proteins.txt",sep="\t", header=None)
proteins_of_interest.columns = ["Protein"] 
our_proteins = raw[raw["Protein"].isin(proteins_of_interest["Protein"] ) ]
### LD matrix was created on the RAP on variants defined in paper
ld_matrix = pd.read_csv("ld_matrix.ld", delimiter=r"\s+" )
ld_matrix = ld_matrix[["SNP_A", "SNP_B","R2"]].copy() 
our_proteins[["CHR", "BP", "REF" , "ALT"]] = our_proteins.Genotype.str.split("-", expand=True) 
grouping_lists = [ ["stop_gained", "disruptive_inframe_deletion", "disruptive_inframe_insertion", "frameshift_variant", "conservative_inframe_deletion", "splice_acceptor_variant", "start_lost", "splice_donor_variant"], ["missense_variant"] , 
                  ["synonymous_variant"] ,  ["non_coding_transcript_exon_variant" ], ['3_prime_UTR_variant','5_prime_UTR_variant', 'synonymous_variant&splice_region_variant', "5_prime_UTR_premature_start_codon_gain_variant","missense_variant&splice_region_variant", "stop_retained_variant&splice_region_variant", "5_prime_UTR_truncation", "stop_retained_variant"]] 
names =  ['High_IMPACT', 'MISS', 'SYNONYMOUS', 'NON_CODING', 'OTHER']

var_scores = ["5", "4" , "2", "3", "1"]   


var_class = (pd.DataFrame({'Groups': names, "Most Damaging Effect": grouping_lists , "variant_class_score": var_scores})
        .explode('Most Damaging Effect')
        .reset_index(drop=True))

our_protein1 = our_proteins.merge(var_class,  on = "Most Damaging Effect", how='left').fillna("Other")

In [34]:
proteins_of_interest[proteins_of_interest["Protein"] == "C19ORF12"] 

,Protein
135,C19ORF12


In [41]:
raw[raw["Protein"] == "C19orf12"] 

,Gene,Protein,Match(?Cis),cis_trans_position_1mb_coding_region,Num_Proteins_p<1x10-8,Genotype,Most Damaging Effect,nSamples,nCarriers,nNonCarriers,p-value,beta,lower,upper,se,model,panel,Olink Assay,MAF
53685,PNPLA3,C19orf12,No,"trans-position, trans-gene",96,22-43928847-C-G,missense_variant,44579,16517,28062,4.489905e-30,0.090495,0.074939,0.106051,0.007937,genotypic,Neurology,C19orf12:Q9NSK7:OID20804:v1:Neurology,0.209341
62572,PNPLA3,C19orf12,No,"trans-position, trans-gene",96,22-43928847-C-G,missense_variant,44579,2150,42429,1.265995e-25,0.226142,0.183809,0.268475,0.021598,recessive,Neurology,C19orf12:Q9NSK7:OID20804:v1:Neurology,0.209341
71018,C19orf12,C19orf12,Yes,"cis-position, cis-gene",1,19-29702747-T-C,missense_variant,45354,83,45271,2.102760e-22,-1.045686,-1.256074,-0.835299,0.107340,dominant,Neurology,C19orf12:Q9NSK7:OID20804:v1:Neurology,0.000898
71019,C19orf12,C19orf12,Yes,"cis-position, cis-gene",1,19-29702747-T-C,missense_variant,45354,83,45271,2.102760e-22,-1.045686,-1.256074,-0.835299,0.107340,genotypic,Neurology,C19orf12:Q9NSK7:OID20804:v1:Neurology,0.000898
72643,PON1,C19orf12,No,"trans-position, trans-gene",9,7-95316772-A-T,missense_variant,45041,26626,18415,6.914395e-22,0.064944,0.051712,0.078176,0.006751,genotypic,Neurology,C19orf12:Q9NSK7:OID20804:v1:Neurology,0.362249
80111,PNPLA3,C19orf12,No,"trans-position, trans-gene",96,22-43928847-C-G,missense_variant,44579,16517,28062,8.372943e-20,0.087349,0.068561,0.106137,0.009586,dominant,Neurology,C19orf12:Q9NSK7:OID20804:v1:Neurology,0.209341
80157,PON1,C19orf12,No,"trans-position, trans-gene",5,7-95308134-T-C,missense_variant,45215,22253,22962,8.570174e-20,-0.083754,-0.101774,-0.065734,0.009194,dominant,Neurology,C19orf12:Q9NSK7:OID20804:v1:Neurology,0.288201
80271,PON3,C19orf12,No,"trans-position, trans-gene",8,7-95396288-G-A,synonymous_variant,45167,20492,24675,9.106496e-20,-0.066891,-0.081294,-0.052489,0.007348,genotypic,Neurology,C19orf12:Q9NSK7:OID20804:v1:Neurology,0.261920
80510,SAMM50,C19orf12,No,"trans-position, trans-gene",40,22-43972242-A-G,missense_variant,44307,11977,32330,1.033495e-19,0.083177,0.065241,0.101112,0.009151,genotypic,Neurology,C19orf12:Q9NSK7:OID20804:v1:Neurology,0.147319
80566,PON1,C19orf12,No,"trans-position, trans-gene",5,7-95308134-T-C,missense_variant,45215,22253,22962,1.062281e-19,-0.065073,-0.079110,-0.051036,0.007161,genotypic,Neurology,C19orf12:Q9NSK7:OID20804:v1:Neurology,0.288201


In [14]:
%%time
test_proteins = our_protein1["Protein"].unique()
for i in test_proteins: 
    print (i)
    try:
        temp = our_protein1[our_protein1["Protein"] == i ] 
        temp_clump = normal_clump(temp) 
        score = make_score(temp_clump)
        file_name = "weights_normal_clump/" + i + ".clumpbyp.weights"
        score.to_csv(file_name, sep="\t", index=None) 
    
    except: 
        print ("naughty protein- find out why")
        next

ACE2
Number of variants before clump: 53
Number of variants after clump: 27
ACP5
Number of variants before clump: 71
Number of variants after clump: 35
ACY1
Number of variants before clump: 55
Number of variants after clump: 34
ADA2
Number of variants before clump: 258
Number of variants after clump: 87
ADAM8
Number of variants before clump: 140
Number of variants after clump: 63
ADGRE2
Number of variants before clump: 71
Number of variants after clump: 28
ADGRE5
Number of variants before clump: 85
Number of variants after clump: 28
ADGRG1
Number of variants before clump: 38
Number of variants after clump: 22
ADM
Number of variants before clump: 74
Number of variants after clump: 37
AGR2
Number of variants before clump: 41
Number of variants after clump: 21
ALCAM
Number of variants before clump: 52
Number of variants after clump: 28
ALPP
Number of variants before clump: 122
Number of variants after clump: 49
AMIGO2
Number of variants before clump: 64
Number of variants after clump: 32


In [20]:
our_protein1[our_protein1["Protein"] =="HLA-E"] 

,Gene,Protein,Match(?Cis),cis_trans_position_1mb_coding_region,Num_Proteins_p<1x10-8,Genotype,Most Damaging Effect,nSamples,nCarriers,nNonCarriers,...,model,panel,Olink Assay,MAF,CHR,BP,REF,ALT,Groups,variant_class_score


In [21]:
raw[raw["Protein"] =="HLA-E"] 

,Gene,Protein,Match(?Cis),cis_trans_position_1mb_coding_region,Num_Proteins_p<1x10-8,Genotype,Most Damaging Effect,nSamples,nCarriers,nNonCarriers,p-value,beta,lower,upper,se,model,panel,Olink Assay,MAF
42047,TRIM27,HLA-E,No,"trans-position, trans-gene",79,6-28923399-T-C,synonymous_variant,46241,10294,35947,0.000000e+00,0.372897,0.354149,0.391645,0.009565,genotypic,Inflammation,HLA-E:P13747:OID20532:v1:Inflammation,0.118784
42048,OR12D3,HLA-E,No,"trans-position, trans-gene",89,6-29374998-G-A,missense_variant,46223,10625,35598,0.000000e+00,0.373874,0.355381,0.392368,0.009435,genotypic,Inflammation,HLA-E:P13747:OID20532:v1:Inflammation,0.122719
42049,UBD,HLA-E,No,"cis-position, trans-gene",101,6-29556180-A-G,synonymous_variant,46245,10755,35490,0.000000e+00,0.388539,0.370260,0.406818,0.009326,genotypic,Inflammation,HLA-E:P13747:OID20532:v1:Inflammation,0.124715
42050,HLA-A,HLA-E,No,"cis-position, trans-gene",74,6-29942954-G-A,missense_variant,44601,14646,29955,0.000000e+00,0.315361,0.299753,0.330970,0.007963,genotypic,Inflammation,HLA-E:P13747:OID20532:v1:Inflammation,0.184563
42051,ZNRD1,HLA-E,No,"cis-position, trans-gene",106,6-30064745-A-C,3_prime_UTR_variant,45431,11067,34364,0.000000e+00,0.402805,0.384788,0.420822,0.009192,genotypic,Inflammation,HLA-E:P13747:OID20532:v1:Inflammation,0.131194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42743,HIST1H2BG,HLA-E,No,"trans-position, trans-gene",2,6-26216356-C-T,synonymous_variant,46249,3963,42286,6.760000e-09,-0.089205,-0.119361,-0.059049,0.015386,genotypic,Inflammation,HLA-E:P13747:OID20532:v1:Inflammation,0.044106
42744,MUC22,HLA-E,No,"cis-position, trans-gene",2,6-31026455-A-G,missense_variant,45626,485,45141,7.290000e-09,-0.212990,-0.285150,-0.140829,0.036816,genotypic,Inflammation,HLA-E:P13747:OID20532:v1:Inflammation,0.006029
42745,MUC22,HLA-E,No,"cis-position, trans-gene",2,6-31029498-C-G,missense_variant,46260,482,45778,7.660000e-09,-0.252239,-0.337820,-0.166657,0.043664,genotypic,Inflammation,HLA-E:P13747:OID20532:v1:Inflammation,0.005218
42746,CFB,HLA-E,No,"trans-position, trans-gene",18,6-31950687-A-G,missense_variant,46261,1267,44994,8.660000e-09,-0.155780,-0.208824,-0.102735,0.027063,genotypic,Inflammation,HLA-E:P13747:OID20532:v1:Inflammation,0.013825


In [26]:
proteins_of_interest[proteins_of_interest["Protein"] == "HLA-E"] 

,Protein


In [45]:
i = "C19orf12"
temp = our_protein1[our_protein1["Protein"] == i ] 
temp_clump = normal_clump(temp) 
score = make_score(temp_clump)
file_name = "weights_normal_clump/" + i + ".clumpbyp.weights"
score.to_csv(file_name, sep="\t", index=None) 

Number of variants before clump: 13
Number of variants after clump: 6


In [49]:
i = "HLA-E"
temp = our_protein1[our_protein1["Protein"] == i ] 
temp_clump = normal_clump(temp) 
score = make_score(temp_clump)
file_name = "weights_normal_clump/" + i + ".clumpbyp.weights"
score.to_csv(file_name, sep="\t", index=None) 

Number of variants before clump: 707
Number of variants after clump: 181


In [3]:
new_list = pd.read_csv("get_scores.txt", header=None) 
new_list.head(2)

,0
0,WAS
1,TPP1


In [52]:
for i in new_list[0]: 
    print (i)

WAS
TPP1
AGRN
CA12
CD300C
CD300E
CD302
CD38
CLEC14A
COLEC12
EFNA1
EPHB4
FOLR1
FOLR2
GFRA1
HAVCR2
IFNGR1
IGFBP6
JAM2
KLRB1
LAYN
LGALS9
LRP11
LTBR
MMP7
NBL1
NECTIN2
NECTIN4
RTN4R
SEMA3F
TGFBR2
THBD
TNFRSF19
TNFRSF1A
BAMBI
CLEC5A
ITIH3
MMP9
ITIH3
BAMBI
KLK4
WFIKKN1


In [2]:
### add something
#raw = pd.read_csv("ewas_results_genomodel.txt", sep="\t")
raw = pd.read_excel("exome_gwas.xlsx" ) 
proteins_of_interest = pd.read_csv("get_scores.txt",sep="\t", header=None)
proteins_of_interest.columns = ["Protein"] 
our_proteins = raw[raw["Protein"].isin(proteins_of_interest["Protein"] ) ] 
ld_matrix = pd.read_csv("ld_matrix.ld", delimiter=r"\s+" )
ld_matrix = ld_matrix[["SNP_A", "SNP_B","R2"]].copy() 
our_proteins[["CHR", "BP", "REF" , "ALT"]] = our_proteins.Genotype.str.split("-", expand=True) 
grouping_lists = [ ["stop_gained", "disruptive_inframe_deletion", "disruptive_inframe_insertion", "frameshift_variant", "conservative_inframe_deletion", "splice_acceptor_variant", "start_lost", "splice_donor_variant"], ["missense_variant"] , 
                  ["synonymous_variant"] ,  ["non_coding_transcript_exon_variant" ], ['3_prime_UTR_variant','5_prime_UTR_variant', 'synonymous_variant&splice_region_variant', "5_prime_UTR_premature_start_codon_gain_variant","missense_variant&splice_region_variant", "stop_retained_variant&splice_region_variant", "5_prime_UTR_truncation", "stop_retained_variant"]] 
names =  ['High_IMPACT', 'MISS', 'SYNONYMOUS', 'NON_CODING', 'OTHER']

var_scores = ["5", "4" , "2", "3", "1"]   


var_class = (pd.DataFrame({'Groups': names, "Most Damaging Effect": grouping_lists , "variant_class_score": var_scores})
        .explode('Most Damaging Effect')
        .reset_index(drop=True))

our_protein1 = our_proteins.merge(var_class,  on = "Most Damaging Effect", how='left').fillna("Other")

In [5]:
#our_protein1[our_protein1["Protein"] == "CD38" ] 

In [4]:
%%time
test_proteins = our_protein1["Protein"].unique()
for i in new_list[0]:  
    print (i)
    try:
        temp = our_protein1[our_protein1["Protein"] == i ] 
        temp_clump = normal_clump(temp) 
        score = make_score(temp_clump)
        file_name = "all_protein_weights/" + i + ".clumpbyp.weights"
        score.to_csv(file_name, sep="\t", index=None) 
    
    except: 
        print ("naughty protein- find out why")
        next

WAS
Number of variants before clump: 15
Number of variants after clump: 6
TPP1
Number of variants before clump: 99
Number of variants after clump: 44
AGRN
Number of variants before clump: 95
Number of variants after clump: 90
CA12
Number of variants before clump: 79
Number of variants after clump: 55
CD300C
Number of variants before clump: 88
Number of variants after clump: 49
CD300E
Number of variants before clump: 66
Number of variants after clump: 33
CD302
Number of variants before clump: 294
Number of variants after clump: 114
CD38
Number of variants before clump: 27
Number of variants after clump: 22
CLEC14A
Number of variants before clump: 51
Number of variants after clump: 20
COLEC12
Number of variants before clump: 14
Number of variants after clump: 10
EFNA1
Number of variants before clump: 69
Number of variants after clump: 41
EPHB4
Number of variants before clump: 93
Number of variants after clump: 48
FOLR1
Number of variants before clump: 45
Number of variants after clump: 1

In [15]:
raw = pd.read_excel("exome_gwas.xlsx" ) 
#proteins_of_interest = pd.read_csv("get_scores.txt",sep="\t", header=None)
#proteins_of_interest.columns = ["Protein"] 
our_proteins = raw
ld_matrix = pd.read_csv("ld_matrix.ld", delimiter=r"\s+" )
ld_matrix = ld_matrix[["SNP_A", "SNP_B","R2"]].copy() 
our_proteins[["CHR", "BP", "REF" , "ALT"]] = our_proteins.Genotype.str.split("-", expand=True) 
grouping_lists = [ ["stop_gained", "disruptive_inframe_deletion", "disruptive_inframe_insertion", "frameshift_variant", "conservative_inframe_deletion", "splice_acceptor_variant", "start_lost", "splice_donor_variant"], ["missense_variant"] , 
                  ["synonymous_variant"] ,  ["non_coding_transcript_exon_variant" ], ['3_prime_UTR_variant','5_prime_UTR_variant', 'synonymous_variant&splice_region_variant', "5_prime_UTR_premature_start_codon_gain_variant","missense_variant&splice_region_variant", "stop_retained_variant&splice_region_variant", "5_prime_UTR_truncation", "stop_retained_variant"]] 
names =  ['High_IMPACT', 'MISS', 'SYNONYMOUS', 'NON_CODING', 'OTHER']

var_scores = ["5", "4" , "2", "3", "1"]   


var_class = (pd.DataFrame({'Groups': names, "Most Damaging Effect": grouping_lists , "variant_class_score": var_scores})
        .explode('Most Damaging Effect')
        .reset_index(drop=True))

our_protein1 = our_proteins.merge(var_class,  on = "Most Damaging Effect", how='left').fillna("Other")
our_protein1.head(2)

,Gene,Protein,Match(?Cis),cis_trans_position_1mb_coding_region,Num_Proteins_p<1x10-8,Genotype,Most Damaging Effect,nSamples,nCarriers,nNonCarriers,...,model,panel,Olink Assay,MAF,CHR,BP,REF,ALT,Groups,variant_class_score
0,PLA2G2A,PLA2G2A,Yes,"cis-position, cis-gene",3,1-19978364-G-A,non_coding_transcript_exon_variant,45775,18188,27587,...,dominant,Cardiometabolic,PLA2G2A:P14555:OID20274:v1:Cardiometabolic,0.224092,1,19978364,G,A,NON_CODING,3
1,PLA2G2A,PLA2G2A,Yes,"cis-position, cis-gene",3,1-19978433-G-A,synonymous_variant,45808,18195,27613,...,dominant,Cardiometabolic,PLA2G2A:P14555:OID20274:v1:Cardiometabolic,0.223906,1,19978433,G,A,SYNONYMOUS,2


In [16]:
our_protein1.Protein.unique()

array(['PLA2G2A', 'FUCA1', 'PCSK9', ..., 'RUVBL1', 'STX8', 'GH2'],
      dtype=object)

In [17]:
test_proteins = our_protein1["Protein"].unique()
test_proteins.shape

(1387,)

In [8]:
%%time
test_proteins = our_protein1["Protein"].unique()
for i in our_protein1.Protein.unique():  
    print (i)
    try:
        temp = our_protein1[our_protein1["Protein"] == i ] 
        temp_clump = normal_clump(temp) 
        score = make_score(temp_clump)
        file_name = "all_protein_weights/" + i + ".clumpbyp.weights"
        score.to_csv(file_name, sep="\t", index=None) 
    
    except: 
        print ("naughty protein- find out why")
        next

PLA2G2A
Number of variants before clump: 28
Number of variants after clump: 25
FUCA1
Number of variants before clump: 61
Number of variants after clump: 45
PCSK9
Number of variants before clump: 60
Number of variants after clump: 39
ANGPTL3
Number of variants before clump: 37
Number of variants after clump: 25
AMY2A
Number of variants before clump: 61
Number of variants after clump: 45
AMY2B
Number of variants before clump: 57
Number of variants after clump: 43
IL6R
Number of variants before clump: 92
Number of variants after clump: 57
ADAM15
Number of variants before clump: 189
Number of variants after clump: 142
FCRL1
Number of variants before clump: 111
Number of variants after clump: 52
FCGR2A
Number of variants before clump: 92
Number of variants after clump: 58
FCGR3B
Number of variants before clump: 123
Number of variants after clump: 59
SELP
Number of variants before clump: 56
Number of variants after clump: 36
CHIT1
Number of variants before clump: 38
Number of variants after 

### what the ld matrix looks like -> done in plink

In [2]:
ld_matrix = pd.read_csv("ld_matrix.ld", delimiter=r"\s+" )
ld_matrix = ld_matrix[["SNP_A", "SNP_B","R2"]].copy() 

In [6]:
ld_matrix.head(2)

,SNP_A,SNP_B,R2
0,1:1197697:A:G,1:1203822:T:C,0.192521
1,1:1197697:A:G,1:1204118:C:T,0.025365
